In [1]:
!pip install openai

  Using cached jiter-0.5.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.3/361.3 kB 15.7 MB/s eta 0:00:00
Using cached jiter-0.5.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (318 kB)


In [2]:
!pip uninstall pydantic -y

Found existing installation: pydantic 1.10.16
Uninstalling pydantic-1.10.16:
  Successfully uninstalled pydantic-1.10.16


In [3]:
!pip install pydantic==1.10.8

  Using cached pydantic-1.10.8-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (146 kB)
Using cached pydantic-1.10.8-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.7.3
    Uninstalling pydantic-2.7.3:
      Successfully uninstalled pydantic-2.7.3


In [4]:
from openai import OpenAI
import pandas as pd
import numpy as np
import re
import os
from dotenv import load_dotenv

In [5]:
from collections import Counter
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt', quiet=True)

from nltk.corpus import stopwords

nltk.download('stopwords', quiet=True)

True

In [6]:
load_dotenv()

client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY")
)

# Sample text

In [7]:
wiki_content_path = '../wiki_content.csv'
wiki_content_df = pd.read_csv(wiki_content_path)

In [16]:
section_test = wiki_content_df.sample()
text_test = section_test['content'].values[0]

In [54]:
text_test

'Spatial inference, or estimation, of a quantity Z : : R n → → R {\\displaystyle Z\\colon \\mathbb {R} ^{n}\\to \\mathbb {R} } , at an unobserved location x 0 {\\displaystyle x_{0}} , is calculated from a linear combination of the observed values z i = Z ( x i ) {\\displaystyle z_{i}=Z(x_{i})} and weights w i ( x 0 ) , i = 1 , … … , N {\\displaystyle w_{i}(x_{0}),\\;i=1,\\ldots ,N} : The weights w i {\\displaystyle w_{i}} are intended to summarize two extremely important procedures in a spatial inference process: reflect the structural "proximity" of samples to the estimation location x 0 {\\displaystyle x_{0}} ; at the same time, they should have a desegregation effect, in order to avoid bias caused by eventual sample clusters. When calculating the weights w i {\\displaystyle w_{i}} , there are two objectives in the geostatistical formalism: unbias and minimal variance of estimation. If the cloud of real values Z ( x 0 ) {\\displaystyle Z(x_{0})} is plotted against the estimated value

## Paraphrasing Agent

In [21]:
few_shot_para_examples = [
        {
            "text": "Linear discriminant analysis (LDA), normal discriminant analysis (NDA), or discriminant function analysis is a generalization of Fisher's linear discriminant, a method used in statistics and other fields, to find a linear combination of features that characterizes or separates two or more classes of objects or events.",
            "answer": ["A statistical technique known as linear discriminant analysis (LDA), also referred to as normal discriminant analysis (NDA) or discriminant function analysis, expands upon Fisher's linear discriminant method. This approach is employed in various disciplines to identify a linear combination of attributes that distinguishes or defines multiple categories of objects or occurrences."]
        },
        {
            "text": "Linear discriminant analysis (LDA), normal discriminant analysis (NDA), or discriminant function analysis is a generalization of Fisher's linear discriminant, a method used in statistics and other fields, to find a linear combination of features that characterizes or separates two or more classes of objects or events.",
            "answer": ["Originating from Fisher's linear discriminant, linear discriminant analysis (LDA) - also called normal discriminant analysis (NDA) or discriminant function analysis - is a statistical methodology utilized across different fields. Its primary purpose is to determine a linear blend of characteristics that effectively differentiates or describes two or more groups of entities or phenomena."]
        },
        {
            "text": "Linear discriminant analysis (LDA), normal discriminant analysis (NDA), or discriminant function analysis is a generalization of Fisher's linear discriminant, a method used in statistics and other fields, to find a linear combination of features that characterizes or separates two or more classes of objects or events.",
            "answer": ["As an extension of Fisher's linear discriminant, linear discriminant analysis (LDA), which is also known as normal discriminant analysis (NDA) or discriminant function analysis, is a statistical approach employed in numerous domains. This method aims to discover a linear amalgamation of features capable of characterizing or segregating multiple classes of items or events."]
        }
    ]


In [ ]:
def plagiariser_agent_para(text_test):
    """Paraphrasing: Restating the original text in different words while maintaining the same meaning and level of detail."""
    few_shot_prompt = "Here are some examples of how to paraphrase:\n\n"
    for example in few_shot_para_examples:
        few_shot_prompt += f"Text: {example['text']}\n"
        few_shot_prompt += f"Answer: {example['answer']}\n\n"

    full_prompt = (
        f"{few_shot_prompt}"
        f"Now, paraphrase the following text:\n"
        f"{text_test}\n"
        "Return the results as a Python list."
    )
    
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You paraphrase text and respond in Python lists."},
            {"role": "user", "content": full_prompt}
        ]
    )
    
    answer = completion.choices[0].message.content
    
    try:
        parsed_answer = eval(answer)
        if isinstance(parsed_answer, list):
            return parsed_answer
        else:
            return [answer]
    except:
        return [answer]

In [23]:
plagiariser_agent_para(text_test)

['Spatial inference involves estimating a quantity Z at an unobserved location x0 by using a linear combination of observed values Z(xi) and weights wi(x0). These weights play a crucial role in summarizing two key aspects in spatial inference: reflecting sample proximity to the estimation location x0 and preventing bias due to sample clustering. In geostatistical formalism, the objectives for calculating weights are to ensure unbiased estimates and minimize estimation variance. Meeting the criteria of global unbias and field stationarity involves ensuring the mean of estimations matches the mean of real values, and minimizing the mean of squared deviations between estimated and real values indicates precision.']

In [34]:
def analyze_paraphrases(text_test):
    """
    Generate multiple paraphrases of the input text and analyze common unigrams.

    Args:
    text_test (str): The text to be paraphrased and analyzed.

    Returns:
    list: Common unigrams present in all generated paraphrases.
    """
    stop_words = set(stopwords.words('english'))
    all_unigrams = []
    
    for _ in range(3):
        paraphrase = plagiariser_agent_para(text_test)
        paraphrase = ' '.join(paraphrase) if isinstance(paraphrase, list) else paraphrase
        tokens = word_tokenize(paraphrase.lower())
        unigrams = [word for word in tokens if word.isalnum() and word not in stop_words]
        all_unigrams.append(unigrams)
    
    common_unigrams = set(all_unigrams[0])
    for unigram_list in all_unigrams[1:]:
        common_unigrams.intersection_update(unigram_list)
    
    return list(common_unigrams)

In [35]:
analyze_paraphrases(text_test)

['linear',
 'spatial',
 'geostatistical',
 'sample',
 'unbiased',
 'variance',
 'include',
 'bias',
 'quantity',
 'combination',
 'global',
 'objectives',
 'location',
 'proximity',
 'z',
 'weights',
 'involves',
 'real',
 'values',
 'unobserved',
 'estimating',
 'inference',
 'observed']

## Rewriting Agent

In [43]:
few_shot_rewrite_examples = [
        {#Rewrite (Simplified and Expanded):
            "text": "Linear discriminant analysis (LDA), normal discriminant analysis (NDA), or discriminant function analysis is a generalization of Fisher's linear discriminant, a method used in statistics and other fields, to find a linear combination of features that characterizes or separates two or more classes of objects or events.",
            "answer": ["Fisher's linear discriminant laid the groundwork for a powerful statistical technique known as Linear Discriminant Analysis (LDA). This method, also referred to as Normal Discriminant Analysis (NDA) or Discriminant Function Analysis, has found applications beyond statistics, reaching into various scientific and analytical fields. At its core, LDA seeks to identify a unique blend of characteristics that can effectively distinguish between multiple groups of items or occurrences. By doing so, it provides a valuable tool for classification and data interpretation across diverse disciplines."]
        },
        {#Rewrite (Restructured and Technical Focus):
            "text": "Linear discriminant analysis (LDA), normal discriminant analysis (NDA), or discriminant function analysis is a generalization of Fisher's linear discriminant, a method used in statistics and other fields, to find a linear combination of features that characterizes or separates two or more classes of objects or events.",
            "answer": ["In the realm of multivariate statistics, Linear Discriminant Analysis (LDA) stands out as a versatile classification algorithm. Evolving from Fisher's original linear discriminant, LDA—also known as Normal Discriminant Analysis (NDA) or Discriminant Function Analysis—operates by projecting data onto a lower-dimensional space. Its primary objective is to maximize the separation between different classes while minimizing the spread within each class. This is achieved by identifying an optimal linear combination of features, making LDA an invaluable tool in pattern recognition, machine learning, and data mining across various scientific domains."]
        },
        {#Rewrite (Practical Application Emphasis):
            "text": "Linear discriminant analysis (LDA), normal discriminant analysis (NDA), or discriminant function analysis is a generalization of Fisher's linear discriminant, a method used in statistics and other fields, to find a linear combination of features that characterizes or separates two or more classes of objects or events.",
            "answer": ["Imagine you're tasked with sorting a mixed basket of fruits into distinct categories based on their characteristics. This is essentially what Linear Discriminant Analysis (LDA) does, but with complex datasets. As an extension of Fisher's linear discriminant, LDA—sometimes called Normal Discriminant Analysis (NDA) or Discriminant Function Analysis—is a statistical method that has found its way into numerous fields beyond pure mathematics. It works by finding the best way to combine different features of your data, creating a sort of 'super characteristic' that can tell different groups apart. Whether you're in biology studying gene expression, in finance predicting market trends, or in image processing distinguishing objects, LDA provides a powerful means of classification and analysis."]
        }
    ]

In [38]:
def plagiariser_agent_rewrite(text_test):
    """Rewriting: Altering the original text more significantly, which may involve restructuring, simplifying, or expanding on the content."""
    few_shot_prompt = "Here are some examples of how to rewrite:\n\n"
    for example in few_shot_rewrite_examples:
        few_shot_prompt += f"Text: {example['text']}\n"
        few_shot_prompt += f"Answer: {example['answer']}\n\n"

    full_prompt = (
        f"{few_shot_prompt}"
        f"Now, rewrite the following text:\n"
        f"{text_test}\n"
        "Return the results as a Python list."
    )
    
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You rewrite text and respond in Python lists."},
            {"role": "user", "content": full_prompt}
        ]
    )
    
    answer = completion.choices[0].message.content
    
    try:
        parsed_answer = eval(answer)
        if isinstance(parsed_answer, list):
            return parsed_answer
        else:
            return [answer]
    except:
        return [answer]

In [ ]:
def analyze_rewrites(text_test):
    """
    Generate multiple rewrites of the input text and analyze common unigrams.

    Args:
    text_test (str): The text to be rewritten and analyzed.

    Returns:
    list: Common unigrams present in all generated rewrites.
    """
    stop_words = set(stopwords.words('english'))
    all_unigrams = []
    
    for _ in range(3):
        rewrite = plagiariser_agent_rewrite(text_test)
        rewrite = ' '.join(rewrite) if isinstance(rewrite, list) else rewrite
        tokens = word_tokenize(rewrite.lower())
        unigrams = [word for word in tokens if word.isalnum() and word not in stop_words]
        all_unigrams.append(unigrams)
    
    common_unigrams = set(all_unigrams[0])
    for unigram_list in all_unigrams[1:]:
        common_unigrams.intersection_update(unigram_list)
    
    return list(common_unigrams)

In [42]:
analyze_rewrites(text_test)

['estimation',
 'spatial',
 'also',
 'deviations',
 'geostatistical',
 'ensuring',
 'unbiased',
 'variance',
 'quantity',
 'global',
 'realm',
 'samples',
 'process',
 'location',
 'proximity',
 'z',
 'weights',
 'involves',
 'real',
 'values',
 'estimator',
 'minimal',
 'observed']

## Summarising Agent

In [44]:
few_shot_summary_examples = [
        {#Concise Summary:
            "text": "Linear discriminant analysis (LDA), normal discriminant analysis (NDA), or discriminant function analysis is a generalization of Fisher's linear discriminant, a method used in statistics and other fields, to find a linear combination of features that characterizes or separates two or more classes of objects or events.",
            "answer": ["LDA, also known as NDA or discriminant function analysis, is a statistical method that extends Fisher's linear discriminant to classify or distinguish between multiple groups based on their features."]
        },
        {#Technical Summary:
            "text": "Linear discriminant analysis (LDA), normal discriminant analysis (NDA), or discriminant function analysis is a generalization of Fisher's linear discriminant, a method used in statistics and other fields, to find a linear combination of features that characterizes or separates two or more classes of objects or events.",
            "answer": ["Linear Discriminant Analysis (LDA) generalizes Fisher's linear discriminant technique. It's a statistical method used across various fields to identify linear feature combinations that optimally characterize or separate multiple classes of objects or events."]
        },
        {#Contextual Summary:
            "text": "Linear discriminant analysis (LDA), normal discriminant analysis (NDA), or discriminant function analysis is a generalization of Fisher's linear discriminant, a method used in statistics and other fields, to find a linear combination of features that characterizes or separates two or more classes of objects or events.",
            "answer": ["LDA, NDA, or discriminant function analysis builds upon Fisher's linear discriminant. This statistical approach, applicable in numerous disciplines, aims to find the best linear combination of features for differentiating between two or more categories of items or occurrences."]
        }
    ]

In [ ]:
def plagiariser_agent_summary(text_test):
    """Summarisation: captures the essence of the original text while presenting the information in a more condensed form."""
    few_shot_prompt = "Here are some examples of how to summarise:\n\n"
    for example in few_shot_summary_examples:
        few_shot_prompt += f"Text: {example['text']}\n"
        few_shot_prompt += f"Answer: {example['answer']}\n\n"

    full_prompt = (
        f"{few_shot_prompt}"
        f"Now, summarise the following text:\n"
        f"{text_test}\n"
        "Return the results as a Python list."
    )
    
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You summarise text and respond in Python lists."},
            {"role": "user", "content": full_prompt}
        ]
    )
    
    answer = completion.choices[0].message.content
    
    try:
        parsed_answer = eval(answer)
        if isinstance(parsed_answer, list):
            return parsed_answer
        else:
            return [answer]
    except:
        return [answer]

In [ ]:
def analyze_summaries(text_test):
    """
    Generate multiple summaries of the input text and analyze common unigrams.

    Args:
    text_test (str): The text to be summarized and analyzed.

    Returns:
    list: Common unigrams present in all generated summaries.
    """
    stop_words = set(stopwords.words('english'))
    all_unigrams = []
    
    for _ in range(3):
        summary = plagiariser_agent_summary(text_test)
        summary = ' '.join(summary) if isinstance(summary, list) else summary
        tokens = word_tokenize(summary.lower())
        unigrams = [word for word in tokens if word.isalnum() and word not in stop_words]
        all_unigrams.append(unigrams)
    
    common_unigrams = set(all_unigrams[0])
    for unigram_list in all_unigrams[1:]:
        common_unigrams.intersection_update(unigram_list)
    
    return list(common_unigrams)

In [50]:
analyze_summaries(text_test)

['linear',
 'estimation',
 'spatial',
 'deviations',
 'unbiased',
 'quantity',
 'combination',
 'global',
 'using',
 'unbias',
 'equal',
 'squared',
 'location',
 'proximity',
 'z',
 'weights',
 'involves',
 'real',
 'estimations',
 'values',
 'unobserved',
 'estimating',
 'x0',
 'inference',
 'mean',
 'minimal',
 'observed']

## Keyword Detection

In [51]:
def keywords():
    """
    Extract common keywords across paraphrases, rewrites, and summaries of the global text_test.

    Returns:
    list: Keywords common to all three text manipulation methods.
    """
    paraphrase_keywords = analyze_paraphrases(text_test)
    rewrite_keywords = analyze_rewrites(text_test)
    summary_keywords = analyze_summaries(text_test)
    
    paraphrase_set = set(paraphrase_keywords)
    rewrite_set = set(rewrite_keywords)
    summary_set = set(summary_keywords)
    
    final_keywords = list(paraphrase_set.intersection(rewrite_set, summary_set))
    
    return final_keywords

In [52]:
correct_list = keywords()

In [53]:
correct_list

['location',
 'linear',
 'estimation',
 'spatial',
 'quantity',
 'combination',
 'proximity',
 'values',
 'unobserved',
 'z',
 'mean',
 'inference',
 'weights',
 'observed']

## Synonym Agent

Traditional NLP methods proved unsuccessful in detecting synonyms in the dataset. This is primarily because NLP libraries struggle to identify multi-word terms as one term.

The best solution proved to be using an open-source LLM with few-shot prompts as in-context learning to identify the alternate labels for each word.

In [23]:
wiki_content_latest = '../plagiarism_source.csv'
wiki_content = pd.read_csv(wiki_content_latest)
wiki_content.head()

,wiki_path,title,h2,h3,content
0,cluster_analysis,cluster analysis,Introduction,NaN,cluster analysis or clustering is the task of ...
1,cluster_analysis,cluster analysis,Definition,NaN,"the notion of a ""cluster"" cannot be precisely ..."
2,cluster_analysis,cluster analysis,Algorithms,Connectivity-based clustering (hierarchical cl...,"connectivity-based clustering, also known as h..."
3,cluster_analysis,cluster analysis,Algorithms,Centroid-based clustering,"in centroid-based clustering, each cluster is ..."
4,cluster_analysis,cluster analysis,Algorithms,Model-based clustering,the clustering framework most closely related ...


## Test Results

In [29]:
intro_rows = wiki_content[wiki_content['h2'] == 'Introduction']

unique_titles = intro_rows['title'].unique()

num_titles = min(3, len(unique_titles))
random_titles = np.random.choice(unique_titles, size=num_titles, replace=False)

result = intro_rows[intro_rows['title'].isin(random_titles)].groupby('title').first().reset_index()


In [30]:
result.content

0    fuzzy clustering (also referred to as soft clu...
1    a generative adversarial network ( gan ) is a ...
2    synthetic data is information that is artifici...
Name: content, dtype: object

In [31]:
few_shot_synonym_examples = [
        {
            "title": "Linear Discriminant Analysis",
            "text": "Linear discriminant analysis (LDA), normal discriminant analysis (NDA), or discriminant function analysis is a generalization of Fisher's linear discriminant, a method used in statistics and other fields, to find a linear combination of features that characterizes or separates two or more classes of objects or events.",
            "answer": ["Linear Discriminant Analysis", "LDA", "Normal Discriminant Analysis", "NDA", "Discriminant Function Analysis"]
        },
        {
            "title": "Canonical correlation",
            "text": "In statistics, canonical-correlation analysis (CCA), also called canonical variates analysis, is a way of...",
            "answer": ["CCA", "Canonical variates analysis"]
        },
        {
            "title": "Non-negative matrix factorization",
            "text": "Non-negative matrix factorization (NMF or NNMF), also non-negative matrix approximation[1][2] is a group of algorithms in multivariate analysis and linear algebra where...",
            "answer": ["Non-negative matrix factorization", "NMF", "NNMF"]
        },
        {
            "title": "Support vector machine",
            "text": "In machine learning, support vector machines (SVMs, also support vector networks[1]) is a method...",
            "answer": ["Support vector machine", "SVM", "Support vector networks"]
        }
    ]

In [ ]:
def extract_synonyms_initialisms(df):
    """
    Extract synonyms and initialisms for the title from the content of a DataFrame row.

    Uses GPT-3.5-turbo to identify synonyms and initialisms, then categorizes them.

    Args:
    df (pd.DataFrame): A DataFrame row containing 'title' and 'content' columns.

    Returns:
    pd.DataFrame: The input DataFrame row with added 'synonyms' and 'acronyms' columns.
    """
    title = df['title']
    text = df['content']
    
    few_shot_prompt = "Here are some examples of identifying synonyms and initialisms:\n\n"
    for example in few_shot_synonym_examples:
        few_shot_prompt += f"Title: {example['title']}\n"
        few_shot_prompt += f"Text: {example['text']}\n"
        few_shot_prompt += f"Answer: {example['answer']}\n\n"

    full_prompt = (
        f"{few_shot_prompt}"
        f"Now, identify any synonyms or initialisms for '{title}' from the following text:\n"
        f"{text}\n"
        "Return the results as a Python list."
    )

    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You detect synonyms and initialisms from text and respond in Python lists."},
            {"role": "user", "content": full_prompt}
        ]
    )
    answer = completion.choices[0].message.content
    
    try:
        parsed_answer = eval(answer)
        if isinstance(parsed_answer, list):
            synonyms = []
            acronyms = []
            for item in parsed_answer:
                if item.lower() == title.lower():
                    continue
                elif item.isupper() or re.search(r'[A-Z]{3,}', item):
                    acronyms.append(item)
                else:
                    synonyms.append(item)
            
            df['synonyms'] = synonyms if synonyms else None
            df['acronyms'] = acronyms if acronyms else None
        else:
            df['synonyms'] = None
            df['acronyms'] = None
    except:
        df['synonyms'] = None
        df['acronyms'] = None
    
    return df

In [39]:
result_with_synonyms = result.apply(extract_synonyms_initialisms, axis=1)

['Fuzzy clustering', 'Soft clustering', 'Soft k-means']
['Generative Adversarial Network', 'GAN', 'Machine Learning Frameworks', 'Generative AI']
['Artificial data', 'Artificially generated data', 'Generated data', 'Algorithmically generated data']


In [40]:
result_with_synonyms

,title,wiki_path,h2,h3,content,synonyms,acronyms
0,fuzzy clustering,fuzzy_clustering,Introduction,None,fuzzy clustering (also referred to as soft clu...,"[Soft clustering, Soft k-means]",None
1,generative adversarial network,generative_adversarial_network,Introduction,None,a generative adversarial network ( gan ) is a ...,"[Machine Learning Frameworks, Generative AI]",[GAN]
2,synthetic data,synthetic_data,Introduction,None,synthetic data is information that is artifici...,"[Artificial data, Artificially generated data,...",None
